In [22]:
import numpy as np
import netCDF4 as NC
import matplotlib.pyplot as plt
import glob
import sisl

In [23]:
def readNC(fn,ispin=0,nbands=20):
    nc = NC.Dataset(fn)
    kx = nc.variables['grid'][:,0]
    ev = nc.variables['eigenvalues'][:,ispin,-nbands:]
    nc.close()
    return kx,ev

def plot(fn):
    plt.plot()

In [73]:
def analyse(geom,t1,t2,t3):
    #global backbone
    molecule = sisl.get_sile(geom)
    mol = molecule.read_geom()
    mol.sc.set_nsc([3,1,1])
    #mol = mol.translate( -mol.center(what='xyz') )
    # mol.atom[0] = sisl.Atom(1,R=1.2) # how to set a radius!!!
    Hlist = []
    for ia in mol:
        if mol.atoms[ia].Z==1:
            Hlist.append(ia)
    backbone = mol.remove(Hlist)
    H = sisl.Hamiltonian(backbone)
    # Define radii for 1NN, 2NN, 3NN etc
    Rsph = [0.1,1.6,2.6,3.1,3.5]
    bondlengths = []
    for ia in backbone:
        idx,rij = backbone.close(ia,R=Rsph,ret_rij=True)
        bondlengths += [r for r in rij[1:]]
        H[ia,idx[0]] = 0.0
        H[ia,idx[1]] = -t1 # C-C bond 2.7
        if third:
            H[ia,idx[2]] = -t2 # C-C bond 0.2
            H[ia,idx[3]] = -t3 # C-C bond 0.18
    return H

def TBbands(geom,t1,t2,t3):
    H = analyse(geom,t1,t2,t3)
    klist = np.linspace(0,0.5,101)
    eigs = np.empty([len(klist), H.no])
    for ik, k in enumerate(klist):
        eigs[ik,:] = H.eigh([k,0,0], eigvals_only=True)
    return klist,eigs

In [116]:
def mkplots(t1=2.7,t2=0.2,t3=0.18):
    if t2 > 0 or t3 > 0:
        tb = '3NN'
    else:
        tb = '1NN'

    # Read structure from here:
    ka,evtb = TBbands('LDA/31-ribbon/8.870/molecule.XV',t1,t2,t3)
    ka = 2*ka # Units ka/pi
    egap = (evtb[0,14]+evtb[0,13])/2 # determine midgap

    for xc in [r'LDA', r'GGA']:
        if 'LDA' in xc:
            fn = xc+'/31-ribbon/8.870/Ebands/Electrons.nc' # LDA minimum
        else:
            fn = xc+'/31-ribbon/8.970/Ebands/Electrons.nc' # GGA minimum
        fig = plt.figure(figsize=(4,6));
        axes = plt.axes();

        # Add siesta bands
        k1,ev = readNC(fn)
        egap2 = (ev[0,9]+ev[0,10])/2
        plt.plot(ka,ev-egap2,'k');
        # Add TB results
        plt.plot(ka, evtb-egap, 'r')
        plt.ylim(-4,4)
        plt.rc('font', family='Bitstream Vera Serif', size=16)
        plt.rc('text', usetex=True)

        axes.set_title(xc+r' vs.~'+tb)
        axes.set_xlabel(r'$ka/\pi$')
        axes.set_ylabel(r'$E_{nk}$ (eV)')
        #plt.xticks(np.arange(0,.6,0.1))
        plt.subplots_adjust(left=0.2)
        plt.savefig(xc+'-'+tb+'-%.2f-%.2f-%.2f.pdf'%(t1,t2,t3))
        plt.close('all')


In [118]:
for t in [2.7,2.8,2.9]:
    mkplots(t1=t, t2=0.0, t3=0.00)
    mkplots(t1=t, t2=0.2, t3=0.18)

In [117]:
mkplots(t1=2.9, t2=0.0, t3=0.0)

In [84]:
mkplots(t1=2.9, t2=0.2, t3=0.18)